In [1]:
from nltk.corpus import movie_reviews

In [2]:
movie_reviews

<CategorizedPlaintextCorpusReader in '/Users/aayushkhurana/nltk_data/corpora/movie_reviews'>

In [3]:
import nltk
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/aayushkhurana/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [26]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/aayushkhurana/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [27]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/aayushkhurana/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [6]:
movie_reviews.categories()

['neg', 'pos']

In [120]:
#This dataset contains many files which are divided into two categoriesd : Negative and Positive movie reviews
#To load all the files we can use fileids
movie_reviews.fileids()
len(movie_reviews.fileids())

2000

In [7]:
#So this dataset contains 2000 files of which 1000 are positive reviews files and 1000 are negative reviews file
#To get only negative review files we can pass the category 
movie_reviews.fileids('neg')
len(movie_reviews.fileids('neg'))


1000

In [8]:
#To get all the words of any file we can use 
movie_reviews.words(movie_reviews.fileids()[5])
#Here we have listed all the words in the fifth file 


['capsule', ':', 'in', '2176', 'on', 'the', 'planet', ...]

In [9]:
#Code to make a list of tuples of the type : [('Words of file 1','category of file 1')]
documents=[]
for category in movie_reviews.categories():
    for fileid in movie_reviews.fileids(category):
        documents.append((movie_reviews.words(fileid),category))
documents[:5]

[(['plot', ':', 'two', 'teen', 'couples', 'go', 'to', ...], 'neg'),
 (['the', 'happy', 'bastard', "'", 's', 'quick', 'movie', ...], 'neg'),
 (['it', 'is', 'movies', 'like', 'these', 'that', 'make', ...], 'neg'),
 (['"', 'quest', 'for', 'camelot', '"', 'is', 'warner', ...], 'neg'),
 (['synopsis', ':', 'a', 'mentally', 'unstable', 'man', ...], 'neg')]

In [10]:
#So we have 1000 negative tuples and 1000 positive tuples
#So let us shuffle them using random.shuffle
import random
random.shuffle(documents)
documents[:5]

[(['synopsis', ':', 'original', '"', 'jurassic', 'park', ...], 'neg'),
 (['for', 'timing', 'reasons', 'having', 'to', 'do', ...], 'neg'),
 (['quiz', 'show', ',', 'an', 'almost', 'perfectly', ...], 'pos'),
 (['summary', 'five', 'liberal', 'iowa', 'graduate', ...], 'pos'),
 (['she', 'was', 'the', 'femme', 'in', '"', 'la', ...], 'neg')]

In [22]:
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

In [12]:
lemmatizer=WordNetLemmatizer()


In [87]:
w="aayush"
pos=pos_tag([w])
pos[0][1]

'NN'

In [86]:
#function to return the part of speech required by the lemmatizer to lemmatize
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    

In [20]:
def clean_review(words):
    output_words = []
    for w in words:
        if w.lower() not in stops:
            pos = pos_tag([w])        #pos_tag expects an array , hence we have passed [w] and not just w 
            clean_word = lemmatizer.lemmatize(w,pos=get_simple_pos(pos[0][1]))   #pos array's zeroth row and its first column is retrieved
            output_words.append(clean_word.lower())
    return output_words

In [16]:
#Creating the stopwords list
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
len(stops)

179

In [17]:
#To remove punctuations , we add the list of punctuations to stop words itself
import string
punctuation = set(string.punctuation)
print(len(punctuation))
stops.update(punctuation)
len(stops)

32


211

In [67]:
document_clean=[]
for i in range(len(documents)):
    document_clean.append((clean_review(documents[i][0]),documents[i][1]))

In [68]:
len(document_clean)

2000

In [120]:
train_documents=document_clean[0:1500]
test_documents=document_clean[1500:]


In [71]:
all_words=[]
for doc in train_documents:
    all_words = all_words + doc[0]       #When two arrays are added the secomd array gets appended to the first

In [72]:
freq = nltk.FreqDist(all_words)
common = freq.most_common(3000)       #To get the 3000 top frequency words 
features = [i[0] for i in common ]     #To store only the top 3000 words without their frequencies

In [73]:
#Now we need to create a dictionary to see if the top frequency word comes in that document or not and return true or false
def get_feature_dict(words):
    current_features={}
    words_set=set(words)   #Just creates a set of words that the function receives as the array-words
    for w in features:
        current_features[w] = w in words_set
    return current_features
    

In [75]:
#Training data tuples of the type ((dictionary stating true or false,category)) for each document 
training_data=[]
for doc in train_documents:
    training_data.append((get_feature_dict(doc[0]),doc[1]))


In [78]:
#Testing data tuples of the type ((dictionary stating true or false,category)) for each document 
testing_data=[]
for doc in test_documents:
    testing_data.append((get_feature_dict(doc[0]),doc[1]))



In [88]:
#We created our data in this format as sklearn nltk classifier requires data to be in this format only .
#What's the format : Array of tuples in which each tuple has a dictionary (with key as the feature and value as true or false)
#and the category that document belongs to
from nltk import NaiveBayesClassifier


In [90]:
#Syntax would be a little different than that we used to follow
classifier = NaiveBayesClassifier.train(training_data)


In [91]:
#Now in nltk we have a accuracy function which requires 2 arguments : our classifier and the testing data
nltk.classify.accuracy(classifier,testing_data)

0.806

In [93]:
classifier.show_most_informative_features(15)

Most Informative Features
               ludicrous = True              neg : pos    =     20.7 : 1.0
             beautifully = True              pos : neg    =     10.8 : 1.0
             outstanding = True              pos : neg    =      9.8 : 1.0
                   anger = True              pos : neg    =      9.7 : 1.0
                  turkey = True              neg : pos    =      8.0 : 1.0
                   mulan = True              pos : neg    =      6.9 : 1.0
             wonderfully = True              pos : neg    =      6.8 : 1.0
                  random = True              neg : pos    =      6.2 : 1.0
                   damon = True              pos : neg    =      6.0 : 1.0
            breathtaking = True              pos : neg    =      6.0 : 1.0
                  seagal = True              neg : pos    =      5.9 : 1.0
              uninspired = True              neg : pos    =      5.8 : 1.0
                   idiot = True              neg : pos    =      5.8 : 1.0

In [95]:
#Now we will use sklkearn classifiers to train our model
#using SVC 
from sklearn .svm import SVC
from nltk.classify.scikitlearn import SklearnClassifier  

In [96]:
svc = SVC()
classifier_sklearn = SklearnClassifier(svc)    #The Sklearnclassifier at the back uses svc object 


In [97]:
classifier_sklearn.train(training_data)

/Users/aayushkhurana/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


<SklearnClassifier(SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))>

In [98]:
nltk.classify.accuracy(classifier_sklearn,testing_data)

0.798

In [99]:
#Now lets use random forest classifier 
from sklearn.ensemble import RandomForestClassifier

In [101]:
rfc = RandomForestClassifier()

In [108]:
classifier_sklearn1 = SklearnClassifier(rfc)    #The Sklearnclassifier at the back uses rfc object 

In [109]:
classifier_sklearn1.train(training_data)

<SklearnClassifier(RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))>

In [110]:
nltk.classify.accuracy(classifier_sklearn1,testing_data)

0.68

In [111]:
#Now to use the sklearn classifier directly (without the help of nltk) we need to connvert the data into the format :
#X is a 2D array of values in which each column is a feature and rows are the frequencies of that feature in a given document
#and Y is a 1D array of target values(categories)

In [112]:
#So lets convert our nltk dataset into sklearn required format of X and Y using CountVectorizer
#First let us apply this for some dummy data
from sklearn.feature_extraction.text import CountVectorizer

In [116]:
train_set={"the sky is blue","the sun is bright"}
count_vec=CountVectorizer(max_features=3)
a = count_vec.fit_transform(train_set)       #Here 'a' is a sparse matrix 
a.todense()

matrix([[0, 1, 1],
        [1, 1, 1]], dtype=int64)

In [117]:
#To see the values of features for the sparse matrix we can use
count_vec.get_feature_names()

['blue', 'is', 'the']

In [118]:
" ".join(train_set)

'the sun is bright the sky is blue'

In [147]:
#This example shown above clearly explains that our CountVectorizer requires data in the form of sentences. Hence we join our documents dataset like :
text_documents = [" ".join(doc) for doc,category in document_clean]

In [148]:
categories = [cat for doc,cat in document_clean]      #Y has to be 1D dimensinal target values

In [149]:
from sklearn.model_selection import train_test_split

In [150]:
x_train,x_test,y_train,y_test=train_test_split(text_documents,categories,random_state=1)

In [250]:
count_vec1 = CountVectorizer(max_features=2000,ngram_range=(1,3),max_df=0.8,min_df=0.05)   #ngram_range specifies combination of words (Here 1 and 2 combinations are allowed)

In [251]:
x_train_features = count_vec1.fit_transform(x_train)

In [252]:
x_train_features.todense()

matrix([[0, 0, 0, ..., 1, 0, 1],
        [0, 0, 0, ..., 1, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 1, 0, 1]], dtype=int64)

In [253]:
#count_vec1.get_feature_names()

In [254]:
x_test_features = count_vec1.transform(x_test)

In [255]:
x_test_features

<500x1046 sparse matrix of type '<class 'numpy.int64'>'
	with 67336 stored elements in Compressed Sparse Row format>

In [256]:
svc1=SVC()

In [257]:
svc1.fit(x_train_features,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [258]:
svc1.score(x_test_features,y_test)

0.81

In [ ]:
#N-grams are basically the combination of words. combination of 2 words are called bigrams , 3 are called trigrams and so on
#For eg: 'Not good' is a bigram and 'Not very Good' is a trigram
#N-gram is a property of CountVectorizer and is specified as count_vec1 = CountVectorizer(max_features=2000,ngram_range=(2,3))
#This means only bigrams and trigrams will be our features 

#TF/IDF stands for Term Frequency and Inverse Document Frequency. IDF = 1/TF
#For a word , it's document frequency is defined as the no of documents that contain that word. The point here is that if a word 
#is present in a lot of documents,it's not that important. We like to work with those words that have low document frequency.
#Ideally,a word that comes in few documents can help in classification way better than a word that comes in all the documents.
#To implement this we have two more parameters in CountVectorizer : max_df and min_df 
#max_df = 0.8 means if a word is present in 80% of the documents , I am not interested in that word.
#min_df = 0.1 means if a word is not present in 10% of the documents , I am not interested in that word.
#max_df kind of helps us remove stopwords from our vocabulary
#One interesting thing is that if a word like e-mail is present in a lot of documents and is not a stopword , then it's document frequency will be very high.






In [260]:
from sklearn.neural_network import MLPClassifier

In [313]:
mlp=MLPClassifier(hidden_layer_sizes=(50,50,), max_iter=300000000)

In [314]:
mlp.fit(x_train_features,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(50, 50), learning_rate='constant',
       learning_rate_init=0.001, max_iter=300000000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [315]:
mlp.score(x_test_features,y_test)

0.832